In [2]:
class ArrayStack(object):
    def __init__ (self):
        self._data = []
        
    def __len__ (self):
        return len(self._data)
    
    def is_empty(self):
        return len(self._data) == 0
    
    def push(self, e):
        self._data.append(e)
        
    def top(self):
        if self.is_empty( ):
            raise ValueError( 'Stack is empty' )
        return self._data[-1]
    def pop(self):
        if self.is_empty( ):
            raise ValueError( 'Stack is empty' )
        return self._data.pop()
    def printstack(self):
        if self.is_empty( ):
            raise ValueError( 'Stack is empty' )
        for i in range(len(self._data)):
            print(self._data[i], end = ' ')
        print()
        

### <a id='Ex1'>Ex.1 Implement Queue using Stacks </a>

In [3]:
class QueueWithTwoStacks:
    def __init__(self):
        self.insertStack = []
        self.popStack = []
        
    def enqueue(self, e):
        self.insertStack.append(e)
        return e
        
    def dequeue(self): 
        if len(self.insertStack) == 0 and len(self.popStack) == 0:
            return None
        if len(self.popStack) == 0:
            while len(self.insertStack) != 0:
                self.popStack.append(self.insertStack.pop())
        return self.popStack.pop()

mystack = QueueWithTwoStacks()
e = mystack.enqueue(3)
print(e)
e = mystack.enqueue(2)
print(e)
e = mystack.enqueue(1)
print(e)
e = mystack.dequeue()
print(e)
e = mystack.dequeue()
print(e)

3
2
1
3
2


### <a id='Ex2'>Ex.2 Stack Sorting</a>

Write a program to sort a stack in ascending order. You should not make any assumptions about how the stack is implemented. The following are the only functions that should be used to write this program: push | pop | peek | isEmpty.

In [3]:
def sortStack(s):
    r = ArrayStack()
    
    while s.is_empty() == False:
        tmp = s.pop()
        while r.is_empty() == False and r.top() > tmp:
            s.push(r.pop())
        r.push(tmp)
        
    return r

mystack = ArrayStack()
print ('size was: ', str(len(mystack)))
mystack.push(3)
mystack.push(1)
mystack.push(4)
mystack.push(2)
mystack.push(5)
mystack.push(6)
mystack.push(9)
mystack.push(8)
mystack.push(7)
mystack.printstack()

r = sortStack(mystack)
r.printstack()

size was:  0
3 1 4 2 5 6 9 8 7 
1 2 3 4 5 6 7 8 9 


### <a id='Ex3'>Ex.3 Valid Parentheses</a>

Given a string containing just the characters '(', ')', '{', '}', '[' and ']', determine if the input string is valid.

# Solution：
[()] √
([)] ×
我们发现括号里面要么为空要么为完整的括号，这就符合了相邻抵消的特性，那么这就适用于stack。

当出现a左括号，可以将其push进stack，当遇到a右括号的时候再将其pop出来。如果遇到([)]则会报错。

In [6]:
def isValid(string):
    if string == None:
        return True
    stack = []
    for s in string:
        if s == '(' or s == '[' or s == '{':
            stack.append(s)
        if ((s == ')' and stack[-1] == '(') # 错误点：把stack写成了s
           or (s == ']' and stack[-1] == '[')
           or (s == '}' and stack[-1] == '{')):
            stack.pop()
    return len(stack) == 0
# 易错点：对空list使用l == None 会返回False，因此判断一个list是否为空最好还是使用len()
s = ""
print(isValid(s))
s = "{}"
print(isValid(s))
s = "{{}}"
print(isValid(s))
s = "{{}}{}{}"
print(isValid(s))
s = "{{{}}{}{{}}}"
print(isValid(s))
s = "[]{}([{}]{})"
print(isValid(s))

True
True
True
True
True
True


In [7]:
a = []
print(a == None)

False


### <a id='Ex3'>Ex.4 Simplify Path</a>

Given an absolute path for a file (Unix-style), simplify it.

For example,

path = "/home/" => "/home"

path = "/a/./b/../../c/" => "/c"

# Solution:
1. // or /./ 停留在原来的目录
2. /../ 如果不是根目录，则退一级
3. 其他字符则是push进去

In [10]:
def simplifyPath(path):
    stack = []
    splits = path.split('/')
    
    for s in splits:
        if s == '' or s == '.':
            continue
        elif s == '..':
            if len(stack) == 0:
                continue
            else:
                stack.pop()
        else:
            stack.append(s)
    
    return '/' + '/'.join(stack)

path = "/home/"
print(simplifyPath(path))
path = "/a/./b/../../c/d/../e/f/g/../"
print(simplifyPath(path))

/home
/c/e/f


### <a id='Ex5'>Ex.5 Decode String</a>

Given an encoded string, return it's decoded string.

The encoding rule is: k[encoded_string], where the encoded_string inside the square brackets is being repeated exactly k times. Note that k is guaranteed to be a positive integer.

You may assume that the input string is always valid; No extra white spaces, square brackets are well-formed, etc.

Furthermore, you may assume that the original data does not contain any digits and that digits are only for those repeat numbers, k. For example, there won't be input like 3a or 2[4].

Examples:

s = "3[a]2[bc]", return "aaabcbc".

s = "3[a2[c]]", return "accaccacc".

s = "2[abc]3[cd]ef", return "abcabccdcdcdef".

# Solution:
这个题目属于嵌套的类型，优先需要解决最里面括号的内容，因此可以使用stack
1. 往stack中添加的数据结构为pair，[string，nums]
2. 当遇到数字的时候先计算nums，这里要考虑数字是非个位数的情况，因此需要读取[前的所有数字
3. 当遇到 [ 的时候放入['', nums]
4. 遇到string的时候，将其存入st中
5. 当遇到 ] 的时候，对stack进行pop，根据nums和st，将st重复nums次并push进stack

In [13]:
def decodeString(s):
    stack = []
    stack.append(['', 1]) # 遗漏点：要先push进一层因为当执行第一个]的时候，pop完stack为空，stack[-1][0]会报错
    nums = ''
    for ch in s:
        if ch.isdigit():
            nums += ch
        elif ch == '[':
            stack.append(['', int(nums)])
            nums = '' # 易错点：这里一定要更新nums，因为不更新的话前面的nums会一直加进后面的nums中
        elif ch == ']':
            st, n = stack.pop()
            stack[-1][0] += st * n
        else:
            stack[-1][0] += ch
    return stack[0][0]

s = "3[a]2[bc]"
print(decodeString(s))
s = "3[a2[c]]"
print(decodeString(s))
s = "2[abc]3[cd]ef"
print(decodeString(s))
s = "12[ab]"
print(decodeString(s))

aaabcbc
accaccacc
abcabccdcdcdef
abababababababababababab


### <a id='Ex6'>Ex.6 Basketball Game</a>

You're now a baseball game point recorder.

Given a list of strings, each string can be one of the 4 following types:

- Integer (one round's score): Directly represents the number of points you get in this round.
- "+" (one round's score): Represents that the points you get in this round are the sum of the last two valid round's points.
- "D" (one round's score): Represents that the points you get in this round are the doubled data of the last valid round's points.
- "C" (an operation, which isn't a round's score): Represents the last valid round's points you get were invalid and should be removed.

Each round's operation is permanent and could have an impact on the round before and the round after.

You need to return the sum of the points you could get in all the rounds.

Input: ["5","2","C","D","+"] 

Output: 30 

- Round 1: You could get 5 points. The sum is: 5. 

- Round 2: You could get 2 points. The sum is: 7. Operation 1: The round 2's data was invalid. The sum is: 5. 

- Round 3: You could get 10 points (the round 2's data has been removed). The sum is: 15. 

- Round 4: You could get 5 + 10 = 15 points. 

- The sum is: 30.


# Solution
类似于计分型的题目都可以使用stack
1. 遇到数字，push进stack
2. 遇到C，pop
3. 遇到+，pop -》 peek -》double -》push*2
4. 遇到D，peek * 2 -》 push

In [14]:
def calPoints(ops):
    total = 0
    stack = []
    for op in ops:
        if op.isdigit():
            stack.append(int(op))
        elif op == 'C':
            stack.pop()
        elif op == '+':
            stack.append(stack[-1] + stack[-2])
        elif op == 'D':
            stack.append(stack[-1] * 2)
        
    total = sum(stack)
    return total

ops = ["5","2","C","D","+"]
calPoints(ops)

30

### <a id='Ex7'>Ex.7  Asteroid Collision</a>

We are given an array asteroids of integers representing asteroids in a row.

For each asteroid, the absolute value represents its size, and the sign represents its direction (positive meaning right, negative meaning left). Each asteroid moves at the same speed.

Find out the state of the asteroids after all collisions. If two asteroids meet, the smaller one will explode. If both are the same size, both will explode. Two asteroids moving in the same direction will never meet.

Example 1:

- Input: asteroids = [5, 10, -5] 
- Output: [5, 10] 
- Explanation: The 10 and -5 collide resulting in 10. The 5 and 10 never collide. 

Example 2:

- Input: asteroids = [10, 2, -5] 
- Output: [10] 
- Explanation: The 2 and -5 collide resulting in -5. The 10 and -5 collide resulting in 10. 

Example 3:

- Input: asteroids = [-2, -1, 1, 2]
- Output: [-2, -1, 1, 2]
- Explanation: The -2 and -1 are moving left, while the 1 and 2 are moving right.
- Asteroids moving the same direction never meet, so no asteroids will meet each other.


# Solution
我们可以发现如果数组的左侧是负数那就不会发生碰撞，而当正数右方出现负数的时候就会发生碰撞。

1. 先创建一个stack，
2. 如果stack为空时push，
3. 当遇到正数的时候进行push，
4. 当遇到负数的时候循环直到此负数被吞并或者栈空
    - 判断栈顶是否为正数
        - True: 将cur与top进行对比，如果cur>top，pop(), 如果相等，pop -> continue， 否则 continue
        - False: continue
    

In [3]:
def asteroidCollision(asteroids):
    if len(asteroids) == 0:
        return []
    stack = []
    
    for asteroid in asteroids:
        if asteroid > 0:
            stack.append(asteroid)
        elif asteroid == 0:
            continue
        else:
            if stack[-1] < 0:
                stack.append(asteroid)
            else:
                while stack[-1] > 0 and stack is not None:
                    if abs(asteroid) > stack[-1]:
                        stack.pop()
                    elif abs(asteroid) == stack[-1]:
                        stack.pop
                        break
                    else:
                        break
                
    return stack

asteroids = [5, 10, -5]
asteroids = [10, 2, -5] 
asteroidCollision(asteroids)
                

[10]

In [ ]:
def asteroidCollision(asteroids):
    ans = []
    for new in asteroids:
        while ans and new < 0 < ans[-1]:
            if ans[-1] < -new:
                ans.pop()
                continue
            elif ans[-1] == -new:
                ans.pop()
            break
        else: # while 可以和 else 一起用
            ans.append(new)
    return ans

### <a id='Ex8'>Ex.8 Next greater element I</a>
给定一个数组，输出每个元素的下一个更大元素。数字 x 的下一个更大的元素是按数组遍历顺序，这个数字之后的第一个比它更大的数，这意味着你应该循环地搜索它的下一个更大的数。如果不存在，则输出 -1。
输入: [1,2,1]
输出: [2,-1,-1]

# Solution
我们发现当输入一个数字且这个数字比前面数字大的时候才可能对前面的数字进行处理。
1. 创建一个stack
2. nums[0] -> stack
3. while nums[1:]:
    - while num > stack[-1] and stack is not none:
        - pop()
    - else
        - push()
        

In [8]:
def nextGreat(nums):
    if len(nums) == 0:
        return []
    stack = []
    stack.append([nums[0], 0]) # [element's value, element's index]
    result = [0] * len(nums)
    
    for i in range(1, len(nums)):
        while len(stack) != 0 and nums[i] > stack[-1][0]:
            cur = stack.pop()
            result[cur[1]] = nums[i]
        else:
            stack.append([nums[i], i])
    for pair in stack:
        result[pair[1]] = -1
    
    return result

array = [6, 4, 5, 2, 15]
nextGreat(array)

[15, 5, 15, 15, -1]

In [19]:
def nextGreat(nums):
    if len(nums) == 0:
        return []
    stack = []
    result = [-1] * len(nums)
    
    for i in range(len(nums)):
        while len(stack) != 0 and nums[i] > nums[stack[-1]]:
            result[stack.pop()] = nums[i]
        else:
            stack.append(i)
    
    return result

array = [6, 4, 5, 2, 15]
nextGreat(array)

[15, 5, 15, 15, -1]

### <a id='Ex9'>Ex.9 Next greater element II</a>
给定一个循环数组（最后一个元素的下一个元素是数组的第一个元素），输出每个元素的下一个更大元素。数字 x 的下一个更大的元素是按数组遍历顺序，这个数字之后的第一个比它更大的数，这意味着你应该循环地搜索它的下一个更大的数。如果不存在，则输出 -1。

输入: [1,2,1]

输出: [2,-1,2]

解释: 第一个 1 的下一个更大的数是 2；

数字 2 找不到下一个更大的数； 

第二个 1 的下一个最大的数需要循环搜索，结果也是 2。

In [13]:
def nextGreat2(nums):
    if len(nums) == 0:
        return []
    stack = []
    stack.append([nums[0], 0]) # [element's value, element's index]
    result = [-1] * len(nums)
    
    for i in range(1, len(nums)):
        while len(stack) != 0 and nums[i] > stack[-1][0]:
            cur = stack.pop()
            result[cur[1]] = nums[i]
        else:
            stack.append([nums[i], i])
    
    for i in range(0, len(nums)):
        while len(stack) != 0 and nums[i] > stack[-1][0]:
            cur = stack.pop()
            result[cur[1]] = nums[i]
        if stack == []:
            break
    
    return result

array = [16, 4, 5, 2, 15]
nextGreat2(array)

[-1, 5, 15, 15, 16]

### <a id='Ex10'>Ex.10 Daily Temperature</a>
Given a list of daily temperatures, produce a list that, for each day in the input, tells you how many days you would have to wait until a warmer temperature. If there is no future day for which this is possible, put 0 instead.

For example, given the list temperatures = [73, 74, 75, 71, 69, 72, 76, 73], your output should be [1, 1, 4, 2, 1, 1, 0, 0].

In [22]:
def dailyTemperature(temperatures):
    if len(temperatures) == 0:
        return []
    stack = []
    result = [0] * len(temperatures)
    
    for i in range(len(temperatures)):
        while len(stack) != 0 and temperatures[i] > temperatures[stack[-1]]:
            cur = stack.pop()
            result[cur] = i - cur
        else:
            stack.append(i)
            
    return result

t = [73, 74, 75, 71, 69, 72, 76, 73]
dailyTemperature(t)

[1, 1, 4, 2, 1, 1, 0, 0]